# Preparing data

In [1]:
corpus = [
    'Human machine interface for Lab ABC computer applications',
    'A survey of user opinion of computer system response time',
    'The EPS user interface management system',
    'System and human system engineering testing of EPS',
    'Relation of user-perceived response time to error measurement',
    'The generation of random, binary, unordered trees',
    'The intersection graph of paths in trees',
    'Graph minors IV: Width of trees and well-quasi-ordering',
    'Graph minors: A survey'
]
stoplist = set('for a of the and to'.split())

In [2]:
# creating generator object for streaming tweets
class Tweets:
    def __iter__(self):
        for tweet in corpus:
            yield tweet

In [3]:
# streaming corpus and storing documents in bow representation
import re
from collections import defaultdict

tweets = Tweets()
# token2id : dict of (str, int)
#   token -> tokenId
token2id = {}
# idf: dict of (int, int)
#   tokenId -> how many instances of this token are contained in the documents
idf = defaultdict(int)
docs2bow = []
for docno, tweet in enumerate(tweets):
    document = re.sub(r'[-,:.]', ' ', tweet.lower()).split()
    counter = defaultdict(int)
    for word in document:
        if word in stoplist: continue
        if word not in token2id: token2id[word] = len(token2id)
        counter[word] += 1
        idf[token2id[word]] += 1
    doc2bow = [(token2id[word], freq) for word, freq in counter.items()]
    print(docno, doc2bow)
    docs2bow.append(doc2bow)

0 [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]
1 [(7, 1), (8, 1), (9, 1), (5, 1), (10, 1), (11, 1), (12, 1)]
2 [(13, 1), (8, 1), (2, 1), (14, 1), (10, 1)]
3 [(10, 2), (0, 1), (15, 1), (16, 1), (13, 1)]
4 [(17, 1), (8, 1), (18, 1), (11, 1), (12, 1), (19, 1), (20, 1)]
5 [(21, 1), (22, 1), (23, 1), (24, 1), (25, 1)]
6 [(26, 1), (27, 1), (28, 1), (29, 1), (25, 1)]
7 [(27, 1), (30, 1), (31, 1), (32, 1), (25, 1), (33, 1), (34, 1), (35, 1)]
8 [(27, 1), (30, 1), (7, 1)]


In [4]:
token2id

{'human': 0,
 'machine': 1,
 'interface': 2,
 'lab': 3,
 'abc': 4,
 'computer': 5,
 'applications': 6,
 'survey': 7,
 'user': 8,
 'opinion': 9,
 'system': 10,
 'response': 11,
 'time': 12,
 'eps': 13,
 'management': 14,
 'engineering': 15,
 'testing': 16,
 'relation': 17,
 'perceived': 18,
 'error': 19,
 'measurement': 20,
 'generation': 21,
 'random': 22,
 'binary': 23,
 'unordered': 24,
 'trees': 25,
 'intersection': 26,
 'graph': 27,
 'paths': 28,
 'in': 29,
 'minors': 30,
 'iv': 31,
 'width': 32,
 'well': 33,
 'quasi': 34,
 'ordering': 35}

In [5]:
len(idf)

36

In [6]:
idf

defaultdict(int,
            {0: 2,
             1: 1,
             2: 2,
             3: 1,
             4: 1,
             5: 2,
             6: 1,
             7: 2,
             8: 3,
             9: 1,
             10: 4,
             11: 2,
             12: 2,
             13: 2,
             14: 1,
             15: 1,
             16: 1,
             17: 1,
             18: 1,
             19: 1,
             20: 1,
             21: 1,
             22: 1,
             23: 1,
             24: 1,
             25: 3,
             26: 1,
             27: 3,
             28: 1,
             29: 1,
             30: 2,
             31: 1,
             32: 1,
             33: 1,
             34: 1,
             35: 1})

In [7]:
len(docs2bow)

9

In [8]:
docs2bow

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)],
 [(7, 1), (8, 1), (9, 1), (5, 1), (10, 1), (11, 1), (12, 1)],
 [(13, 1), (8, 1), (2, 1), (14, 1), (10, 1)],
 [(10, 2), (0, 1), (15, 1), (16, 1), (13, 1)],
 [(17, 1), (8, 1), (18, 1), (11, 1), (12, 1), (19, 1), (20, 1)],
 [(21, 1), (22, 1), (23, 1), (24, 1), (25, 1)],
 [(26, 1), (27, 1), (28, 1), (29, 1), (25, 1)],
 [(27, 1), (30, 1), (31, 1), (32, 1), (25, 1), (33, 1), (34, 1), (35, 1)],
 [(27, 1), (30, 1), (7, 1)]]

In [9]:
# filter once words
bad_ids = set(tokenid for tokenid, freq in idf.items() if freq == 1)
token2id = {token: tokenid for token, tokenid in token2id.items() if idf[tokenid] > 1}
idf = {tokenid: freq for tokenid, freq in idf.items() if freq > 1}

In [10]:
bad_ids

{1,
 3,
 4,
 6,
 9,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 26,
 28,
 29,
 31,
 32,
 33,
 34,
 35}

In [11]:
token2id

{'human': 0,
 'interface': 2,
 'computer': 5,
 'survey': 7,
 'user': 8,
 'system': 10,
 'response': 11,
 'time': 12,
 'eps': 13,
 'trees': 25,
 'graph': 27,
 'minors': 30}

In [12]:
idf

{0: 2, 2: 2, 5: 2, 7: 2, 8: 3, 10: 4, 11: 2, 12: 2, 13: 2, 25: 3, 27: 3, 30: 2}

In [13]:
idmap = dict(zip(sorted(token2id.values()), range(len(token2id))))
idmap

{0: 0,
 2: 1,
 5: 2,
 7: 3,
 8: 4,
 10: 5,
 11: 6,
 12: 7,
 13: 8,
 25: 9,
 27: 10,
 30: 11}

In [14]:
# note this cell is one time run
token2id = {token: idmap[tokenid] for token, tokenid in token2id.items()}
idf = {idmap[tokenid]: freq for tokenid, freq in idf.items()}

In [15]:
token2id

{'human': 0,
 'interface': 1,
 'computer': 2,
 'survey': 3,
 'user': 4,
 'system': 5,
 'response': 6,
 'time': 7,
 'eps': 8,
 'trees': 9,
 'graph': 10,
 'minors': 11}

In [16]:
idf

{0: 2, 1: 2, 2: 2, 3: 2, 4: 3, 5: 4, 6: 2, 7: 2, 8: 2, 9: 3, 10: 3, 11: 2}

In [17]:
docs2bow

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)],
 [(7, 1), (8, 1), (9, 1), (5, 1), (10, 1), (11, 1), (12, 1)],
 [(13, 1), (8, 1), (2, 1), (14, 1), (10, 1)],
 [(10, 2), (0, 1), (15, 1), (16, 1), (13, 1)],
 [(17, 1), (8, 1), (18, 1), (11, 1), (12, 1), (19, 1), (20, 1)],
 [(21, 1), (22, 1), (23, 1), (24, 1), (25, 1)],
 [(26, 1), (27, 1), (28, 1), (29, 1), (25, 1)],
 [(27, 1), (30, 1), (31, 1), (32, 1), (25, 1), (33, 1), (34, 1), (35, 1)],
 [(27, 1), (30, 1), (7, 1)]]

In [18]:
# rebuild docs2bow
docs2bow = [
    [(idmap[tokenid], docfreq) for tokenid, docfreq in doc2bow if tokenid not in bad_ids]
    for doc2bow in docs2bow
]

In [19]:
docs2bow

[[(0, 1), (1, 1), (2, 1)],
 [(3, 1), (4, 1), (2, 1), (5, 1), (6, 1), (7, 1)],
 [(8, 1), (4, 1), (1, 1), (5, 1)],
 [(5, 2), (0, 1), (8, 1)],
 [(4, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(10, 1), (9, 1)],
 [(10, 1), (11, 1), (9, 1)],
 [(10, 1), (11, 1), (3, 1)]]

In [20]:
# not necessary
# creating words2bod by docs2bow -> n -> n.T -> words2bod
import numpy as np

n = np.zeros((len(docs2bow), len(idf)))
for docno, doc2bow in enumerate(docs2bow):
    for tokenid, docfreq in doc2bow:
        n[docno, tokenid] += docfreq
n

array([[1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 2., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.]])

In [21]:
n.T

array([[1., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 1., 0., 1., 0., 0., 0., 0.],
       [0., 1., 1., 2., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 1.]])

In [22]:
words2bod = [
    [(tokenid, int(docfreq)) for tokenid, docfreq in enumerate(rows) if docfreq != 0]
    for rows in n.T
]

In [23]:
words2bod

[[(0, 1), (3, 1)],
 [(0, 1), (2, 1)],
 [(0, 1), (1, 1)],
 [(1, 1), (8, 1)],
 [(1, 1), (2, 1), (4, 1)],
 [(1, 1), (2, 1), (3, 2)],
 [(1, 1), (4, 1)],
 [(1, 1), (4, 1)],
 [(2, 1), (3, 1)],
 [(5, 1), (6, 1), (7, 1)],
 [(6, 1), (7, 1), (8, 1)],
 [(7, 1), (8, 1)]]

# Creating model

In [24]:
from numpy.random import rand

def random_init_pars(K, nshape):
    N, M = nshape
    Pz = rand(K); Pz /= sum(Pz)
    Pd_z = rand(N, K); Pd_z /= Pd_z.sum(axis=0)
    Pw_z = rand(M, K); Pw_z /= Pw_z.sum(axis=0)
    return Pz, Pd_z, Pw_z

In [25]:
n.shape

(9, 12)

In [26]:
random_init_pars(2, n.shape)

(array([0.85563045, 0.14436955]),
 array([[0.17753205, 0.10028764],
        [0.0808015 , 0.09743664],
        [0.27629938, 0.19340294],
        [0.04852657, 0.17056689],
        [0.25285615, 0.0475193 ],
        [0.01423311, 0.09800138],
        [0.05063532, 0.02517477],
        [0.06940201, 0.16138295],
        [0.0297139 , 0.1062275 ]]),
 array([[0.06501794, 0.11012108],
        [0.01302362, 0.0964885 ],
        [0.0562286 , 0.11160602],
        [0.05286686, 0.0353294 ],
        [0.0856485 , 0.04440034],
        [0.1261308 , 0.08803724],
        [0.04545787, 0.02041228],
        [0.08718833, 0.0335275 ],
        [0.14152878, 0.09731091],
        [0.15296093, 0.09745069],
        [0.05400536, 0.12905325],
        [0.11994241, 0.13626278]]))

In [27]:
def likelihood(pars, docs2bow):
    Pz, Pd_z, Pw_z = pars
    L = 0
    for d, doc2bow in enumerate(docs2bow):
        for w, ndw in doc2bow:
            Pcocur = sum(Pz[:] * Pd_z[d,:] * Pw_z[w, :])
            L += ndw * np.log(Pcocur)
    return L

In [28]:
likelihood(random_init_pars(2, n.shape), docs2bow)

-144.95387988556553

## EM

In [29]:
def Estep(pars, docs2bow):  # no necessity to pass docs2bow (datas) to Estep, but it could decrease computations
    Pz, Pd_z, Pw_z = pars
    poster = np.zeros((len(Pz), len(Pd_z), len(Pw_z))) 
    # poster could be an attribute and no need to reset to zeros because it's not accumulative
    for z in range(len(Pz)):
        for d, doc2bow in enumerate(docs2bow):
            for w, ndw in doc2bow:
                poster[z, d, w] = Pz[z] * Pd_z[d, z] * Pw_z[w, z]
    # normalization
    poster /= poster.sum(axis=0) + 1e-16
    return poster

In [30]:
poster = Estep(random_init_pars(2, n.shape), docs2bow)
poster

array([[[0.34952734, 0.67387495, 0.35002385, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.72335671, 0.28559825, 0.60148453,
         0.64354867, 0.73005858, 0.81745679, 0.        , 0.        ,
         0.        , 0.        ],
        [0.        , 0.76528633, 0.        , 0.        , 0.32908344,
         0.36977364, 0.        , 0.        , 0.31376156, 0.        ,
         0.        , 0.        ],
        [0.41543676, 0.        , 0.        , 0.        , 0.        ,
         0.32965959, 0.        , 0.        , 0.27705264, 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.31572897,
         0.        , 0.45259169, 0.57788246, 0.        , 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.90713565,
  

In [31]:
poster.shape

(2, 9, 12)

In [32]:
poster.sum(axis=0)

array([[1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.]])

In [33]:
def Mstep(poster, docs2bow):
    K, N, M = poster.shape  # K, N, M could be in an attribute self.archit
    rePz, rePd_z, rePw_z = np.zeros(K), np.zeros((N, K)), np.zeros((M, K))  # should reset to zeros because they're accumulative
    for z in range(K):
        for d, doc2bow in enumerate(docs2bow):
            for w, ndw in doc2bow:
                rePz[z] += ndw * poster[z, d, w]
                rePd_z[d, z] += ndw * poster[z, d, w]
                rePw_z[w, z] += ndw * poster[z, d, w]
    # normalization
    rePz /= sum(rePz)
    rePd_z /= rePd_z.sum(axis=0)
    rePw_z /= rePw_z.sum(axis=0)
    repars = rePz, rePd_z, rePw_z
    return repars

In [34]:
pars = random_init_pars(2, n.shape)
print(pars)
print(likelihood(pars, docs2bow))
poster = Estep(pars, docs2bow)
repars = Mstep(poster, docs2bow)
print(repars)
print(likelihood(repars, docs2bow))

(array([0.47279364, 0.52720636]), array([[0.05965265, 0.21361667],
       [0.14755665, 0.22089365],
       [0.1665747 , 0.21332415],
       [0.16299642, 0.05454264],
       [0.00839319, 0.04951006],
       [0.1112006 , 0.09466588],
       [0.12646585, 0.01177785],
       [0.14572789, 0.09634246],
       [0.07143206, 0.04532663]]), array([[0.13913857, 0.12536791],
       [0.05626474, 0.11894068],
       [0.14245509, 0.00952256],
       [0.14640991, 0.10920056],
       [0.06580476, 0.05833382],
       [0.01412209, 0.08044686],
       [0.0517519 , 0.12997643],
       [0.08550539, 0.06545851],
       [0.10164124, 0.00540443],
       [0.08228007, 0.06200849],
       [0.09162302, 0.0914898 ],
       [0.02300321, 0.14384995]]))
-140.3212289613326
(array([0.45838518, 0.54161482]), array([[0.08370607, 0.12015671],
       [0.18618867, 0.22442226],
       [0.13007074, 0.14458344],
       [0.17819787, 0.10385198],
       [0.02777023, 0.16749692],
       [0.04385355, 0.02655197],
       [0.13792513

In [35]:
repars[1].sum(axis=0)

array([1., 1.])

In [36]:
def EMsteps(runtimes, pars, docs2bow):
    print(pars)
    print(likelihood(pars, docs2bow))
    for runtime in range(runtimes):
        poster = Estep(pars, docs2bow)
        pars = Mstep(poster, docs2bow)
        print(likelihood(pars, docs2bow))
    print(pars)

In [37]:
pars = random_init_pars(2, n.shape)
EMsteps(20, pars, docs2bow)

(array([0.00665207, 0.99334793]), array([[0.16493635, 0.04231414],
       [0.10754274, 0.14936025],
       [0.14367906, 0.02901166],
       [0.14501712, 0.01993587],
       [0.0499861 , 0.05188395],
       [0.1110577 , 0.18755638],
       [0.04488477, 0.19542478],
       [0.1231146 , 0.19246755],
       [0.10978156, 0.13204542]]), array([[0.08517213, 0.08497412],
       [0.12402174, 0.13536657],
       [0.07406   , 0.0583317 ],
       [0.05018923, 0.14859388],
       [0.07998341, 0.00764178],
       [0.13704147, 0.1514245 ],
       [0.13036373, 0.0516775 ],
       [0.13867284, 0.04039347],
       [0.06490141, 0.0131511 ],
       [0.00507703, 0.09174674],
       [0.04119675, 0.11556539],
       [0.06932028, 0.10113325]]))
-154.42482432140096
-131.52940772986753
-130.4942373335948
-128.87679091609087
-126.81214896943243
-124.72079117055857
-123.27071016119109
-122.50343022922975
-122.238172122838
-122.15226196165605
-122.08620809852431
-121.98886134893979
-121.8177156559297
-121.49884794

In [38]:
token2id

{'human': 0,
 'interface': 1,
 'computer': 2,
 'survey': 3,
 'user': 4,
 'system': 5,
 'response': 6,
 'time': 7,
 'eps': 8,
 'trees': 9,
 'graph': 10,
 'minors': 11}

In [39]:
corpus

['Human machine interface for Lab ABC computer applications',
 'A survey of user opinion of computer system response time',
 'The EPS user interface management system',
 'System and human system engineering testing of EPS',
 'Relation of user-perceived response time to error measurement',
 'The generation of random, binary, unordered trees',
 'The intersection graph of paths in trees',
 'Graph minors IV: Width of trees and well-quasi-ordering',
 'Graph minors: A survey']

## TEM

In [40]:
def TEstep(beta, pars, docs2bow):
    Pz, Pd_z, Pw_z = pars
    poster = np.zeros((len(Pz), len(Pd_z), len(Pw_z)))
    for z in range(len(Pz)):
        for d, doc2bow in enumerate(docs2bow):
            for w, ndw in doc2bow:
                poster[z, d, w] = Pz[z] * (Pd_z[d, z] * Pw_z[w, z])**beta
    poster /= poster.sum(axis=0) + 1e-16
    return poster

In [41]:
def TMstep(poster, docs2bow):
    K, N, M = poster.shape
    rePz, rePd_z, rePw_z = np.zeros(K), np.zeros((N, K)), np.zeros((M, K))
    for z in range(K):
        for d, doc2bow in enumerate(docs2bow):
            for w, ndw in doc2bow:
                rePz[z] += ndw * poster[z, d, w]
                rePd_z[d, z] += ndw * poster[z, d, w]
                rePw_z[w, z] += ndw * poster[z, d, w]
    rePz /= sum(rePz)
    rePd_z /= rePd_z.sum(axis=0)
    rePw_z /= rePw_z.sum(axis=0)
    repars = rePz, rePd_z, rePw_z
    return repars

In [42]:
def TEMsteps(pars, docs2bow):
    beta = 1
    new_likeli = likelihood(pars, docs2bow)
    likeli = -10000
    counter = 0
    while round(new_likeli, 6) > round(likeli, 6):
        counter += 1
        likeli = new_likeli
        print(counter, likeli)
        poster = TEstep(beta, pars, docs2bow)
        pars = TMstep(poster, docs2bow)
        new_likeli = likelihood(pars, docs2bow)
    print(new_likeli)

In [43]:
def TEMsteps(eta, pars, docs2bow):
    beta = 1
    for i in range(100):
        print(beta)
        pars = random_init_pars(2, n.shape)
        new_likeli = likelihood(pars, docs2bow)
        likeli = -10000
        while new_likeli > likeli:
            likeli = new_likeli
            print(likeli)
            # one iteration TEM
            poster = TEstep(beta, pars, docs2bow)
            pars = TMstep(poster, docs2bow)
            new_likeli = likelihood(pars, docs2bow)
        print(new_likeli)
        # new_likeli = likeli
        beta *= eta

In [44]:
pars = random_init_pars(2, n.shape)
TEMsteps(0.9, pars, docs2bow)

1
-143.8315669463502
-130.42484745359044
-128.08742267296756
-124.64554721545574
-121.81525844147275
-120.49740122731518
-119.55579245308836
-118.43490671018704
-117.18056887670213
-116.38859086078492
-116.16813343936612
-116.0660348139822
-115.96525986209808
-115.88327831377954
-115.8336097430055
-115.80988128393149
-115.80004404914354
-115.79623178249295
-115.79479512916532
-115.79425956203264
-115.79406072294995
-115.7939870126522
-115.79395970345033
-115.79394958768336
-115.7939458409318
-115.79394445322257
-115.79394393925297
-115.79394374889341
-115.79394367838984
-115.79394365227735
-115.7939436426061
-115.7939436390241
-115.7939436376975
-115.79394363720613
-115.79394363702411
-115.79394363695674
-115.7939436369318
-115.79394363692255
-115.7939436369191
-115.79394363691785
-115.79394363691736
-115.79394363691718
-115.79394363691713
-115.79394363691709
-115.7939436369171
0.9
-150.21760335763196
-132.04391306703693
-131.85341942607926
-131.58792516564313
-131.18054097731056
-130.

In [45]:
a = 1